In [70]:
#import packages
import numpy as np
import cv2

In [71]:
#open video
cap = cv2.VideoCapture('/Users/danielrosenthal/Documents/EGR_314/Matt_side3_720.mov')
if cap.isOpened() == False:
    print("Cannot open video file.")

In [72]:
#object detection parameters
object_detector = cv2.createBackgroundSubtractorMOG2(history = 100, varThreshold= 50)

In [73]:
#ellipse detection parameters
params = cv2.SimpleBlobDetector_Params()

#area filtering
params.filterByArea = True
params.minArea = 100

#circle filtering
params.filterByCircularity = True
params.minCircularity = 0.9

#convexity parameters
params.filterByConvexity = True
params.minConvexity = 0.2

#inertia parameters
params.filterByInertia = True
params.minInertiaRatio = 0.01

#create detector
detector = cv2.SimpleBlobDetector_create(params)



In [74]:
while True:
    ret, frame = cap.read()
    
    #region of interest
    height, width, _ = frame.shape
    #print(height, width)
    roi = frame[100:720, 500:1250]
    
    #object detection
    mask = object_detector.apply(roi)
    #cleaning mask
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        #calculate error, remove small elements
        area = cv2.contourArea(cnt)
        if area > 100:
            #cv2.drawContours(roi, [cnt], -1, (0,255,0), 2)
            #drawing rectangle around detected objects
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(roi, (x, y), (x + w, y + h), (0,255,0),3)
    
    keypoints = detector.detect(roi)
    blank = np.zeros((1, 1))
    blobs = cv2.drawKeypoints(roi, keypoints, blank, (0, 0, 255),
                          cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    number_of_blobs = len(keypoints)
    text = "Number of Circular Blobs: " + str(len(keypoints))
    cv2.putText(blobs, text, (20, 550),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 100, 255), 2)

    cv2.imshow("roi", roi)
    cv2.imshow("Frame",frame)
    cv2.imshow("Mask", mask)
    cv2.imshow("Filtering Circular Blobs Only", blobs)

    key = cv2.waitKey(120)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

: 